In [66]:
import os
import bs4
import numpy as np
from langchain_anthropic import ChatAnthropic
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import tiktoken
from langchain.load import loads, dumps
from operator import itemgetter
load_dotenv()
langchain_api = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [67]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(parse_only =bs4.SoupStrainer( class_=("post-content", "post-title", "post-header" )))
)

In [68]:
docs= loader.load()

In [69]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [70]:
out = vectorstore._collection.get(include=["documents", "metadatas"])
print(out.keys())

for i in range(min(5,len(out['ids']))):
    print("ID:", out["ids"][i])
    print("Text:", out["documents"][i][:200])
    print("Meta:", out["metadatas"][i])
    print("="*50)

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])
ID: 910181a3-dee0-44c8-af8f-d3b934ecc9bb
Text: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool con
Meta: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
ID: dc146721-f5f7-418e-82d7-83f8601ea8ff
Text: Memory

Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.
Long-term memory: This provides the agent with th
Meta: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
ID: b767d5e0-ca36-42b0-9fe5-535ba6928c0e
Text: Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a s
Meta: {'source': 'https

In [71]:
vectorstore._collection.count()

126

In [72]:
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_template(
    "Use the context to answer the question.\n\nContext:\n{context}\n\nQuestion: {question}"
)
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the context to answer the question.\n\nContext:\n{context}\n\nQuestion: {question}'), additional_kwargs={})]


In [73]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo',temperature=0)
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is a technique used to break down complex tasks into smaller and simpler steps, making them more manageable. It involves transforming big tasks into multiple manageable tasks and shedding light on the model's thinking process. This can be done through prompting techniques like Chain of Thought (CoT) or Tree of Thoughts, as well as using task-specific instructions or relying on external classical planners."

In [74]:
question = "Do you think Amir is going to become very rich by the age 40 years old?"
document = 'Amir will is going to gain networth of 200 million by the time he is 40 years ols. His income will get to at least 1 million per month.'

In [75]:
encoding = tiktoken.get_encoding("cl100k_base")
len(encoding.encode(question))

18

In [76]:
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question) 
document_result = embd.embed_documents([document])[0]

In [77]:
def cosine(vec1,vec2):
    dot_product = np.dot(vec1,vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [78]:
cosine(query_result,document_result)

np.float64(0.9074394698045489)

In [79]:
prompt = """ Answer the question based on the following contex:
{context}

Question: {question}

"""
prompt = ChatPromptTemplate.from_template(prompt)
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)


In [80]:
chain = prompt | llm | StrOutputParser()
chain.invoke({'context':docs, 'question':'What is Task Decomposition'})

'Task decomposition in a LLM-powered autonomous agent system involves breaking down complex tasks into smaller, more manageable subgoals. This process enables the agent to efficiently handle intricate tasks by dividing them into simpler steps. Task decomposition can be achieved through various methods such as prompting the LLM with specific instructions, using task-specific prompts, or incorporating human inputs. Additionally, external classical planners can be utilized for long-horizon planning, where the LLM translates the problem into a planning domain definition language, requests a classical planner to generate a plan, and then translates the plan back into natural language. This approach outsources the planning step to an external tool, which is common in certain robotic setups.'

In [81]:
rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke('What is Task Decomposition')

'Task decomposition is a technique used by agents to break down complex tasks into smaller and simpler steps. This process allows the agent to better plan and execute the task by dividing it into more manageable components. Techniques like Chain of Thought and Tree of Thoughts are examples of methods that help in task decomposition by breaking down big tasks into multiple smaller tasks or exploring multiple reasoning possibilities at each step.'

Multi Query

In [82]:
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt = ChatPromptTemplate.from_template(template)
generate_queries = (
      prompt
    | ChatOpenAI(model='gpt-3.5-turbo',temperature=0)
    | StrOutputParser()
    | (lambda x:x.split('\n'))
)

def get_unique_union(documents):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]
    

# Testing

# queries = generate_queries.invoke("what is the Decomposition in LLM")
# batches = [retriever.get_relevant_documents(q) for q in queries]
# unique_docs = get_unique_union(batches)

question = "What is task decomposition for LLM agents?"
retriever_chain = generate_queries| retriever.map() | get_unique_union
docs = retriever_chain.invoke({"question":question})

In [83]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = ( {'context': retriever_chain,
                     'question': itemgetter('question')}
                   |prompt
                   |llm|StrOutputParser()
)

final_rag_chain.invoke({'question':question})

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps using techniques such as Chain of Thought (CoT) and Tree of Thoughts. This process helps the agent to utilize more test-time computation and enhance model performance on complex tasks by transforming big tasks into multiple manageable tasks. Additionally, task decomposition for LLM agents can also be achieved through simple prompting, task-specific instructions, or by relying on an external classical planner for long-horizon planning.'

RAG Fusion

In [84]:
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion
    |ChatOpenAI(model = 'gpt-3.5-turbo',temperature=0)
    |StrOutputParser()
    |(lambda x: x.split('\n'))
    
)

# Testing
# results = generate_queries.invoke({'question':"what is decomposition in LLM"})

def reciprocal_rank_fusion(results:list[list],k=60):

    fused_score ={}

    for docs in results:
        for rank,doc in enumerate(docs):
            doc_str = dumps(doc)
            fused_score[doc_str] = fused_score.get(doc_str,0)+1 /(rank+k)

    reranked_items =[(loads(doc),score) for doc ,score in sorted(fused_score.items(), key=lambda x:x[1],reverse=True)]
    return reranked_items

retrieval_chain = (generate_queries
                   |retriever.map()
                   |reciprocal_rank_fusion
)
question = "What is task decomposition for LLM agents?"
docs = retrieval_chain.invoke({'question':question})


template = """
Answer the following question based on this context:

{context}

Question:{question}

"""

prompt = ChatPromptTemplate.from_template(template)

fusion_chain = ( {'context':retrieval_chain, 'question':RunnablePassthrough()}
                |prompt
                |llm
                |StrOutputParser()

)
fusion_chain.invoke(question)

'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps using techniques such as prompting, task-specific instructions, and human inputs. Additionally, there is an approach called LLM+P that involves outsourcing the planning step to an external classical planner, which utilizes the Planning Domain Definition Language (PDDL) to describe the planning problem and generate a plan that is then translated back into natural language.'

Decomposition

In [91]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

generate_queries_decomposition =(
    prompt_decomposition
    |llm
    |StrOutputParser()
    |(lambda x:x.split('\n'))
)
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke(question)


Answer recursively

In [92]:
template = """
here is the question you need to answer:
\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""


decomposition_prompt = ChatPromptTemplate.from_template(template)



In [ ]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


q_a_pairs = ""

for q in questions:
    rag_chain =({
        'context': itemgetter('question') |retriever,
        'question': itemgetter('question'),
        'q_a_pairs': itemgetter('q_a_pairs')}
        |decomposition_prompt
        |llm
        |StrOutputParser()
    )
    answer = rag_chain.invoke({'question':q,'q_a_pairs':q_a_pairs})
    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair